In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, (2,2))
        # x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [4]:
net = Net()

In [5]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [6]:
params = list(net.parameters())

In [7]:
len(params)

10

In [8]:
print(params[0].size())  # conv1's weight

torch.Size([6, 1, 3, 3])


In [10]:
# batch size of 1, channel 1, and 32 x 32
torch.randn(1,1, 32,32)

tensor([[[[ 0.3353,  0.3494,  0.5297,  ...,  0.2619, -0.0481,  0.3171],
          [ 1.0599, -0.0202, -0.0413,  ..., -0.5961,  0.7998, -0.1852],
          [ 1.6045, -0.0400, -0.1557,  ..., -0.3266, -1.6436, -0.4723],
          ...,
          [-0.9004, -0.1799,  0.4314,  ..., -1.4039, -1.7707, -0.3981],
          [ 1.5360, -1.2128,  0.7675,  ..., -1.2788,  0.8411,  1.3659],
          [ 0.0892, -1.5082, -0.1731,  ..., -0.3628, -0.7613, -1.5283]]]])

In [12]:
random_input = torch.randn(1,1,32,32)
out = net(random_input)

In [13]:
out

tensor([[ 0.0009,  0.0322, -0.0254,  0.0644,  0.0512, -0.0187, -0.2023, -0.0670,
         -0.0204, -0.0231]], grad_fn=<AddmmBackward>)

**Zero the gradient buffers, because the optimising operation and backward prop operation are kept separate**

In [14]:
net.zero_grad()

In [15]:
out

tensor([[ 0.0009,  0.0322, -0.0254,  0.0644,  0.0512, -0.0187, -0.2023, -0.0670,
         -0.0204, -0.0231]], grad_fn=<AddmmBackward>)

In [16]:
# we will do backprop on out, and use the current value of out to do so, which we just printed above
out.backward(out)

In [18]:
params[0].size()

torch.Size([6, 1, 3, 3])

In [19]:
params[1].size()

torch.Size([6])

In [20]:
params[1]

Parameter containing:
tensor([-0.0824, -0.3315,  0.0517,  0.0178,  0.2636,  0.2832],
       requires_grad=True)

**Loss**

In [24]:
output = net(random_input)
target = torch.randn(1, 10)   # dummy target
mse_loss = nn.MSELoss()

loss = mse_loss(output, target)

In [25]:
loss

tensor(0.6868, grad_fn=<MseLossBackward>)

In [26]:
loss.grad_fn

In [28]:
loss.grad_fn.next_functions

((<AddmmBackward at 0x11e82f748>, 0),)

In [31]:
# linear
loss.grad_fn.next_functions[0][0]

In [32]:
# relu
loss.grad_fn.next_functions[0][0].next_functions[0][0]

**Backprop**

In [33]:
net.zero_grad()

In [34]:
net.conv1.bias.grad

tensor([0., 0., 0., 0., 0., 0.])

In [35]:
loss.backward()

In [37]:
net.conv1.bias.grad

tensor([ 0.0141,  0.0102,  0.0087,  0.0101,  0.0006, -0.0134])

In [38]:
net.conv1.bias.size()

torch.Size([6])

### updating weights manually

In [39]:
lr = 0.01
for f in net.parameters():
    # weight = weight - learning_rate * gradient
    f.data - f.grad.data * lr

This however has been done for us in `torch.optim` package. With all possible optimisers. 

So, all we gotta do is : 

In [41]:
import torch.optim as optim

# after defining your net
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(random_input)
loss = mse_loss(output, target)
loss.backward()
optimizer.step()        # does the update of weights